# XGBoost model with Airline dataset. Experiment 01

XGBoost regressor with L2 + Airline dataset

In [1]:
import os,sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from lightgbm.sklearn import LGBMRegressor
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from scipy.stats import gmean

from libs.timer import Timer
from libs.loaders import load_fraud, load_iot, load_airline
from libs.conversion import convert_cols_categorical_to_numeric, convert_related_cols_categorical_to_numeric
from libs.conversion import _get_nominal_integer_dict, _convert_to_integer

print("System version: {}".format(sys.version))

System version: 3.6.0 |Anaconda 4.3.1 (64-bit)| (default, Dec 23 2016, 12:22:00) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]


/home/hoaphumanoid/anaconda3/envs/strata/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Load data

In [2]:
%%time
df_plane = load_airline()
print(df_plane.shape)


MOUNT_POINT not found in environment. Defaulting to /fileshare
(115069017, 14)
CPU times: user 1min 40s, sys: 14.6 s, total: 1min 55s
Wall time: 4min 2s


In [3]:
df_plane.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,AA,190,247,SFO,ORD,1846,0,27
1,1987,10,1,4,5,114,EA,57,74,LAX,SFO,337,0,5
2,1987,10,1,4,5,35,HP,351,167,ICT,LAS,987,0,17
3,1987,10,1,4,5,40,DL,251,35,MCO,PBI,142,0,-2
4,1987,10,1,4,8,517,UA,500,208,LAS,ORD,1515,0,17


In [4]:
%%time
df_plane_numeric = convert_related_cols_categorical_to_numeric(df_plane, col_list=['Origin','Dest'])


CPU times: user 1min 49s, sys: 8.25 s, total: 1min 57s
Wall time: 1min 54s


In [5]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,AA,190,247,0,33,1846,0,27
1,1987,10,1,4,5,114,EA,57,74,1,0,337,0,5
2,1987,10,1,4,5,35,HP,351,167,2,4,987,0,17
3,1987,10,1,4,5,40,DL,251,35,3,41,142,0,-2
4,1987,10,1,4,8,517,UA,500,208,4,33,1515,0,17


In [6]:
%%time
df_plane_numeric = convert_cols_categorical_to_numeric(df_plane_numeric, col_list='UniqueCarrier')


CPU times: user 58.3 s, sys: 8.9 s, total: 1min 7s
Wall time: 1min 5s


In [27]:
df_plane.dtypes

Year                  int64
Month                 int64
DayofMonth            int64
DayofWeek             int64
CRSDepTime            int64
CRSArrTime            int64
UniqueCarrier        object
FlightNum             int64
ActualElapsedTime     int64
Origin               object
Dest                 object
Distance              int64
Diverted              int64
ArrDelay              int64
dtype: object

In [7]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,0,190,247,0,33,1846,0,27
1,1987,10,1,4,5,114,1,57,74,1,0,337,0,5
2,1987,10,1,4,5,35,2,351,167,2,4,987,0,17
3,1987,10,1,4,5,40,3,251,35,3,41,142,0,-2
4,1987,10,1,4,8,517,4,500,208,4,33,1515,0,17


In [30]:
df_plane_numeric.head(100).convert_objects()
df_plane_numeric.dtypes

/home/hoaphumanoid/anaconda3/envs/strata/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


Year                 int64
Month                int64
DayofMonth           int64
DayofWeek            int64
CRSDepTime           int64
CRSArrTime           int64
UniqueCarrier        int64
FlightNum            int64
ActualElapsedTime    int64
Origin               int64
Dest                 int64
Distance             int64
Diverted             int64
ArrDelay             int64
dtype: object

In [8]:
#gets a random 80% of the entire set
X_train = df_plane_numeric.sample(frac=0.8, random_state=1)
print(X_train.shape)
#gets the left out portion of the dataset
X_test = df_plane_numeric.loc[~df_plane_numeric.index.isin(X_train.index)]
print(X_test.shape)

(92055214, 14)
(23013803, 14)


### Training

In [9]:
# Variables
KFOLD_SPLITS = 5

In [75]:
def train_xgboost(df, verbose=True):
    if verbose: print("Training data of size {}".format(df.shape))
    x = df.ix[:, df.columns != 'ArrDelay'].as_matrix()
    y = df['ArrDelay'].as_matrix()
    
    n_splits = 5
    skf = StratifiedKFold(n_splits=n_splits, random_state=77, shuffle=True)
    clfs = []
    
    if verbose: print("Computing LightGBM boosted tree using {} kfold cross validation".format(n_splits))
    for train_index, test_index in skf.split(x, y):
        trn_x, val_x = x[train_index,:], x[test_index,:]
        trn_y, val_y = y[train_index], y[test_index]

        
        
        clf = xgb.XGBRegressor(max_depth=6,
                            n_estimators=300,
                            min_child_weight=30,
                            learning_rate=0.01,
                            nthread=20,
                            subsample=0.80,
                            colsample_bytree=0.80,
                            seed=77)
        clf.fit(trn_x, trn_y, eval_set=[(val_x, val_y)], verbose=verbose, eval_metric='rmse', early_stopping_rounds=300)
        clfs.append(clf)
        
    return clfs

In [11]:
X_train_small = X_train.sample(frac=0.001, random_state=1)
print(X_train_small.shape)

(92055, 14)


In [76]:
%%time
clfs = train_xgboost(X_train_small)

Training data of size (92055, 14)
Computing LightGBM boosted tree using 5 kfold cross validation


/home/hoaphumanoid/anaconda3/envs/strata/lib/python3.6/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


[0]	validation_0-logloss:-2.87657
Will train until validation_0-logloss hasn't improved in 300 rounds.
[1]	validation_0-logloss:-27.8727
[2]	validation_0-logloss:-48.5636
[3]	validation_0-logloss:-90.7192
[4]	validation_0-logloss:-126.139
[5]	validation_0-logloss:-168.307
[6]	validation_0-logloss:-190.025
[7]	validation_0-logloss:-205.428
[8]	validation_0-logloss:-216.084
[9]	validation_0-logloss:-227.13
[10]	validation_0-logloss:-233.483
[11]	validation_0-logloss:-243.512
[12]	validation_0-logloss:-248.001
[13]	validation_0-logloss:-250.196
[14]	validation_0-logloss:-251.542
[15]	validation_0-logloss:-252.827
[16]	validation_0-logloss:-254.471
[17]	validation_0-logloss:-254.409
[18]	validation_0-logloss:-254.374
[19]	validation_0-logloss:-253.889
[20]	validation_0-logloss:-253.221
[21]	validation_0-logloss:-254.963
[22]	validation_0-logloss:-255.639
[23]	validation_0-logloss:-255.053
[24]	validation_0-logloss:-252.962
[25]	validation_0-logloss:-252.893
[26]	validation_0-logloss:-252.4

### Evaluation

In [81]:
def compute_prediction(clfs, df, verbose=True):    
    x = df.ix[:, df.columns != 'ArrDelay'].as_matrix() 
    print(x.shape)
    preds = []
    with Timer() as t:
        for clf in clfs:
            preds.append(clf.predict(x))
        pred = gmean(np.array(preds), axis=0)
    if verbose: print("Prediction took %.03f sec.\n" % t.interval) 
    df['prediction'] = pred
    return df

In [84]:
X_test_small = X_test.sample(frac=0.001, random_state=1)
print(X_test_small.shape)

(23014, 14)


In [85]:
X_test_pred = compute_prediction(clfs, X_test_small)
X_test_pred.head(50)

(23014, 13)
Prediction took 4.151 sec.



/home/hoaphumanoid/anaconda3/envs/strata/lib/python3.6/site-packages/scipy/stats/stats.py:314: RuntimeWarning: invalid value encountered in log
  log_a = np.log(a)


,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay,prediction
85782914,2004,9,8,3,636,709,18,6098,24,1,13,47,0,-14,NaN
33917016,1995,5,19,5,1755,2003,11,751,302,50,22,2075,0,8,13.145583
101255940,2006,11,23,4,1035,1255,0,664,177,109,72,1183,0,32,1.094877
93564630,2005,10,16,7,1122,1520,0,730,170,33,43,1182,0,-7,0.967711
16885574,1992,1,6,1,1709,1902,0,1119,220,10,0,1464,0,70,10.572554
13132449,1991,4,9,2,1104,1348,0,570,100,131,10,592,0,-6,0.720562
2477763,1988,3,26,6,2233,2331,6,831,49,62,19,270,0,-8,4.276742
90743270,2005,5,24,2,1145,1235,10,1407,50,90,94,189,0,0,0.468966
33857972,1995,5,15,1,1715,1745,11,1027,85,58,48,426,0,-4,7.065486
51966358,1998,10,30,5,1305,1426,4,2809,76,16,52,391,0,-8,5.448835


In [74]:
min(X_test_pred['prediction'].dropna())

0.042901404